In [19]:
import glob
import os

from matpower import start_instance
from oct2py.core import Oct2PyError

from pypglib import PATH_PYPGLIB

In [3]:
m = start_instance()

In [ ]:
PG = 1  # index for PG
opf_dir = os.path.join(PATH_PYPGLIB, "opf")
mpopt = m.mpoption("verbose", 0, "out.all", 0)
m.push("_mpopt", mpopt)

for file_path in glob.glob(os.path.join(opf_dir, "**", "*.m"), recursive=True)[:3]:
    mpc = m.loadcase(file_path)
    try:
        m.push("_mpc", mpc)
        m.eval("_r1 = runopf(_mpc, _mpopt);", verbose=False)
        sol = {
            "baseMVA": mpc.baseMVA,
            "version": mpc.version,
            "bus": m.eval("_r1.bus;", verbose=False),
            "gen": m.eval("_r1.gen;", verbose=False),
            "branch": m.eval("_r1.branch;", verbose=False),
            "gencost": mpc.gencost,
            "success": m.eval("_r1.success;", verbose=False),
        }
        if sol["success"] == 1:
            sol_ed = m.runpf(sol, mpopt, verbose=False)
            if sol["success"] == 1:
                status = "Success"
                cost = m.totcost(sol["gencost"], sol["gen"][:, [PG]]).sum()
            else:
                status = "Not Converge"
                cost = "--"
        else:
            status = "Infeasible"
            cost = "inf"  # infeasible solution
    except Oct2PyError:
        status = "Error"
        cost = "Err"
    print(file_path)
    print(status)

In [ ]:
# import scipy.io
# import julia
# from julia import Main

# # Initialize Julia and load the required packages
# julia.Julia(compiled_modules=False)
# Main.eval(
# """
# using JuMP, PowerModels, Ipopt
# solver = optimizer_with_attributes(Ipopt.Optimizer, "tol"=>1e-6)
# """
# )

# # List of case files to run (replace with actual .mat file paths or data objects)
# case_files = ["case1.mat", "case2.mat", "case3.mat"]

# # To store the results
# results = []

# # Loop through each case and run the optimization
# for case_file in case_files:
#     # Load the .mat file using scipy.io in Python
#     mat_data = scipy.io.loadmat(case_file)

#     # Pass the .mat data to Julia
#     Main.mat_data = mat_data

#     # Run the AC Optimal Power Flow in Julia and capture the result
#     result = Main.eval('run_ac_opf(mat_data, solver)')

#     # Store the result in the Python results list
#     results.append(result)

# # Now `results` contains the optimization results for each case
# for i, res in enumerate(results):
#     print(f"Result for case {i+1}: {res}")